References: 
Times Series and Encoder/Decoder Code is taken from https://towardsdatascience.com/how-to-use-transformer-networks-to-build-a-forecasting-model-297f9270e630


**Import library and packages.**

In [ ]:
! pip install --quiet "seaborn" "pytorch-lightning>=1.4" "ipython[notebook]" "torch>=1.6, <1.9" "pandas" "torchvision" "torchmetrics>=0.6"

     |████████████████████████████████| 584 kB 10.0 MB/s 
     |████████████████████████████████| 804.1 MB 2.5 kB/s 
     |████████████████████████████████| 409 kB 49.3 MB/s 
     |████████████████████████████████| 136 kB 36.4 MB/s 
     |████████████████████████████████| 596 kB 50.0 MB/s 
     |████████████████████████████████| 1.1 MB 58.7 MB/s 
     |████████████████████████████████| 21.0 MB 76.6 MB/s 
     |████████████████████████████████| 23.2 MB 1.4 MB/s 
     |████████████████████████████████| 23.3 MB 1.4 MB/s 
     |████████████████████████████████| 23.3 MB 1.5 MB/s 
     |████████████████████████████████| 22.1 MB 12.3 MB/s 
     |████████████████████████████████| 22.1 MB 1.3 MB/s 
     |████████████████████████████████| 17.4 MB 298 kB/s 
     |████████████████████████████████| 94 kB 2.9 MB/s 
     |████████████████████████████████| 271 kB 59.7 MB/s 
     |████████████████████████████████| 144 kB 55.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account a

In [ ]:
#imports
import os

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import Linear
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import pytorch_lightning as pl
import cv2

import torchvision.datasets as dset
import torchvision.transforms as T

**Mounting to Drive**

In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = 'CS231n_Milestone'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# This downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd /content/drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
!bash get_datasets.sh
%cd /content/drive/My\ Drive/$FOLDERNAME

Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/My Drive/CS231n_Milestone/cs231n/datasets/'
/content
bash: get_datasets.sh: No such file or directory
/content/drive/.shortcut-targets-by-id/1mgBp8mv16p5XEcQ4fFGUpbAneOwkx-l1/CS231n_Milestone


**Preprocessing Data: Splitting Video in Frames**


In [ ]:
!wget "https://prevention-dataset.uah.es/static/RECORD1/DRIVE1/video_camera1.mp4" --no-check-certificate
!unzip 'video_camera1.mp4'

--2022-05-07 00:18:52--  https://prevention-dataset.uah.es/static/RECORD1/DRIVE1/video_camera1.mp4
Resolving prevention-dataset.uah.es (prevention-dataset.uah.es)... 212.128.69.213
Connecting to prevention-dataset.uah.es (prevention-dataset.uah.es)|212.128.69.213|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 1124225982 (1.0G) [video/mp4]
Saving to: ‘video_camera1.mp4’

video_camera1.mp4   100%[===================>]   1.05G  21.8MB/s    in 76s     

2022-05-07 00:20:09 (14.0 MB/s) - ‘video_camera1.mp4’ saved [1124225982/1124225982]

Archive:  video_camera1.mp4
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of video_camera1.mp4 or
        video_camera1.mp4.zip, 

In [ ]:
'''
cameras = [["https://prevention-dataset.uah.es/static/RECORD1/DRIVE1/video_camera1.mp4"], 
             ["https://prevention-dataset.uah.es/static/RECORD2/DRIVE1/video_camera1.mp4", "https://prevention-dataset.uah.es/static/RECORD2/DRIVE2/video_camera1.mp4"],
             ["https://prevention-dataset.uah.es/static/RECORD3/DRIVE1/video_camera1.mp4", "https://prevention-dataset.uah.es/static/RECORD3/DRIVE2/video_camera1.mp4"],
             ["https://prevention-dataset.uah.es/static/RECORD4/DRIVE1/video_camera1.mp4", "https://prevention-dataset.uah.es/static/RECORD4/DRIVE2/video_camera1.mp4", "https://prevention-dataset.uah.es/static/RECORD4/DRIVE3/video_camera1.mp4"],
             ["https://prevention-dataset.uah.es/static/RECORD5/DRIVE1/video_camera1.mp4", "https://prevention-dataset.uah.es/static/RECORD5/DRIVE2/video_camera1.mp4", "https://prevention-dataset.uah.es/static/RECORD5/DRIVE3/video_camera1.mp4"]]

videos = {}
for camera in range(len(cameras)):
  for video in camera:
    keyVal = "Camera " + str(camera) + ", Video " + str(video)
    videozipped = wget(videoUrls[i])
    videos[keyVal] = unzip(videozipped)



for key, val in videos:

'''
directoryToSave = "/content/drive/MyDrive/CS231n_Milestone/VideoData/Video1Frames"
cap = cv2.VideoCapture('video_camera1.mp4')
i = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==False:
      break
    else: 
      frame = cv2.flip(frame,0)
      nameOfFrame = directoryToSave + str(i) + '.jpg'
      cv2.imwrite(nameOfFrame, frame)
      i+=1
cap.release()

cv2.destroyAllWindows()

In [ ]:
!cd /content/drive/MyDrive/CS231n_Milestone/VideoData

In [ ]:
%cd MyDrive/CS231n_Milestone/VideoData

/content/drive/MyDrive/CS231n_Milestone/VideoData


In [ ]:
!rm -rf Video1Frames/

In [ ]:
!ls -1 | wc -l

23845


In [ ]:
!ls | sort

Streaming output truncated to the last 5000 lines.
Video1Frames5500.jpg
Video1Frames5501.jpg
Video1Frames5502.jpg
Video1Frames5503.jpg
Video1Frames5504.jpg
Video1Frames5505.jpg
Video1Frames5506.jpg
Video1Frames5507.jpg
Video1Frames5508.jpg
Video1Frames5509.jpg
Video1Frames550.jpg
Video1Frames5510.jpg
Video1Frames5511.jpg
Video1Frames5512.jpg
Video1Frames5513.jpg
Video1Frames5514.jpg
Video1Frames5515.jpg
Video1Frames5516.jpg
Video1Frames5517.jpg
Video1Frames5518.jpg
Video1Frames5519.jpg
Video1Frames551.jpg
Video1Frames5520.jpg
Video1Frames5521.jpg
Video1Frames5522.jpg
Video1Frames5523.jpg
Video1Frames5524.jpg
Video1Frames5525.jpg
Video1Frames5526.jpg
Video1Frames5527.jpg
Video1Frames5528.jpg
Video1Frames5529.jpg
Video1Frames552.jpg
Video1Frames5530.jpg
Video1Frames5531.jpg
Video1Frames5532.jpg
Video1Frames5533.jpg
Video1Frames5534.jpg
Video1Frames5535.jpg
Video1Frames5536.jpg
Video1Frames5537.jpg
Video1Frames5538.jpg
Video1Frames5539.jpg
Video1Frames553.jpg
Video1Frames5540.jpg
Video1Fr

In [ ]:
!ls

Streaming output truncated to the last 5000 lines.
Video1Frames12654.jpg  Video1Frames19809.jpg  Video1Frames5501.jpg
Video1Frames12655.jpg  Video1Frames1980.jpg   Video1Frames5502.jpg
Video1Frames12656.jpg  Video1Frames19810.jpg  Video1Frames5503.jpg
Video1Frames12657.jpg  Video1Frames19811.jpg  Video1Frames5504.jpg
Video1Frames12658.jpg  Video1Frames19812.jpg  Video1Frames5505.jpg
Video1Frames12659.jpg  Video1Frames19813.jpg  Video1Frames5506.jpg
Video1Frames1265.jpg   Video1Frames19814.jpg  Video1Frames5507.jpg
Video1Frames12660.jpg  Video1Frames19815.jpg  Video1Frames5508.jpg
Video1Frames12661.jpg  Video1Frames19816.jpg  Video1Frames5509.jpg
Video1Frames12662.jpg  Video1Frames19817.jpg  Video1Frames550.jpg
Video1Frames12663.jpg  Video1Frames19818.jpg  Video1Frames5510.jpg
Video1Frames12664.jpg  Video1Frames19819.jpg  Video1Frames5511.jpg
Video1Frames12665.jpg  Video1Frames1981.jpg   Video1Frames5512.jpg
Video1Frames12666.jpg  Video1Frames19820.jpg  Video1Frames5513.jpg
Video1Frames

**Define Loss Function**

In [ ]:
def smape_loss(y_pred, target):
    loss = 2 * (y_pred - target).abs() / (y_pred.abs() + target.abs() + 1e-8)
    return loss.mean()


def gen_trg_mask(length, device):
    mask = torch.tril(torch.ones(length, length, device=device)) == 1

    mask = (
        mask.float()
        .masked_fill(mask == 0, float("-inf"))
        .masked_fill(mask == 1, float(0.0))
    )

    return mask

**Create Model**

In [ ]:
class TimeSeriesForcasting(pl.LightningModule):
    def __init__(
        self,
        n_encoder_inputs,
        n_decoder_inputs,
        channels=512,
        dropout=0.1,
        lr=1e-4,
    ):
        super().__init__()

        self.save_hyperparameters()

        self.lr = lr
        self.dropout = dropout

        self.input_pos_embedding = torch.nn.Embedding(1024, embedding_dim=channels)
        self.target_pos_embedding = torch.nn.Embedding(1024, embedding_dim=channels)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=channels,
            nhead=8,
            dropout=self.dropout,
            dim_feedforward=4 * channels,
        )
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=channels,
            nhead=8,
            dropout=self.dropout,
            dim_feedforward=4 * channels,
        )

        self.encoder = torch.nn.TransformerEncoder(encoder_layer, num_layers=8)
        self.decoder = torch.nn.TransformerDecoder(decoder_layer, num_layers=8)

        self.input_projection = Linear(n_encoder_inputs, channels)
        self.output_projection = Linear(n_decoder_inputs, channels)

        self.linear = Linear(channels, 1)

        self.do = nn.Dropout(p=self.dropout)

    def encode_src(self, src):
        src_start = self.input_projection(src).permute(1, 0, 2)

        in_sequence_len, batch_size = src_start.size(0), src_start.size(1)
        pos_encoder = (
            torch.arange(0, in_sequence_len, device=src.device)
            .unsqueeze(0)
            .repeat(batch_size, 1)
        )

        pos_encoder = self.input_pos_embedding(pos_encoder).permute(1, 0, 2)

        src = src_start + pos_encoder

        src = self.encoder(src) + src_start

        return src

    def decode_trg(self, trg, memory):

        trg_start = self.output_projection(trg).permute(1, 0, 2)

        out_sequence_len, batch_size = trg_start.size(0), trg_start.size(1)

        pos_decoder = (
            torch.arange(0, out_sequence_len, device=trg.device)
            .unsqueeze(0)
            .repeat(batch_size, 1)
        )
        pos_decoder = self.target_pos_embedding(pos_decoder).permute(1, 0, 2)

        trg = pos_decoder + trg_start

        trg_mask = gen_trg_mask(out_sequence_len, trg.device)

        out = self.decoder(tgt=trg, memory=memory, tgt_mask=trg_mask) + trg_start

        out = out.permute(1, 0, 2)

        out = self.linear(out)

        return out

    def forward(self, x):
        src, trg = x

        src = self.encode_src(src)

        out = self.decode_trg(trg=trg, memory=src)

        return out

    def training_step(self, batch, batch_idx):
        src, trg_in, trg_out = batch

        y_hat = self((src, trg_in))

        y_hat = y_hat.view(-1)
        y = trg_out.view(-1)

        loss = smape_loss(y_hat, y)

        self.log("train_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        src, trg_in, trg_out = batch

        y_hat = self((src, trg_in))

        y_hat = y_hat.view(-1)
        y = trg_out.view(-1)

        loss = smape_loss(y_hat, y)

        self.log("valid_loss", loss)

        return loss

    def test_step(self, batch, batch_idx):
        src, trg_in, trg_out = batch

        y_hat = self((src, trg_in))

        y_hat = y_hat.view(-1)
        y = trg_out.view(-1)

        loss = smape_loss(y_hat, y)

        self.log("test_loss", loss)

        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, patience=10, factor=0.1
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
            "monitor": "valid_loss",
        }

**Generate Fake Data**




In [ ]:
from tqdm import tqdm
from uuid import uuid4
import random

periods = [7, 14, 28, 30]


def get_init_df():

    date_rng = pd.date_range(start="2015-01-01", end="2020-01-01", freq="D")

    dataframe = pd.DataFrame(date_rng, columns=["timestamp"])

    dataframe["index"] = range(dataframe.shape[0])

    dataframe["article"] = uuid4().hex

    return dataframe


def set_amplitude(dataframe):

    max_step = random.randint(90, 365)
    max_amplitude = random.uniform(0.1, 1)
    offset = random.uniform(-1, 1)

    phase = random.randint(-1000, 1000)

    amplitude = (
        dataframe["index"]
        .apply(lambda x: max_amplitude * (x % max_step + phase) / max_step + offset)
        .values
    )

    if random.random() < 0.5:
        amplitude = amplitude[::-1]

    dataframe["amplitude"] = amplitude

    return dataframe


def set_offset(dataframe):

    max_step = random.randint(15, 45)
    max_offset = random.uniform(-1, 1)
    base_offset = random.uniform(-1, 1)

    phase = random.randint(-1000, 1000)

    offset = (
        dataframe["index"]
        .apply(
            lambda x: max_offset * np.cos(x * 2 * np.pi / max_step + phase)
            + base_offset
        )
        .values
    )

    if random.random() < 0.5:
        offset = offset[::-1]

    dataframe["offset"] = offset

    return dataframe


def generate_time_series(dataframe):

    clip_val = random.uniform(0.3, 1)

    period = random.choice(periods)

    phase = random.randint(-1000, 1000)

    dataframe["views"] = dataframe.apply(
        lambda x: np.clip(
            np.cos(x["index"] * 2 * np.pi / period + phase), -clip_val, clip_val
        )
        * x["amplitude"]
        + x["offset"],
        axis=1,
    ) + np.random.normal(
        0, dataframe["amplitude"].abs().max() / 10, size=(dataframe.shape[0],)
    )

    return dataframe


def generate_df():
    dataframe = get_init_df()
    dataframe = set_amplitude(dataframe)
    dataframe = set_offset(dataframe)
    dataframe = generate_time_series(dataframe)
    return dataframe

dataframes = []

for _ in tqdm(range(20000)):
  df = generate_df()
  dataframes.append(df)

  all_data = pd.concat(dataframes, ignore_index=True)

  3%|▎         | 611/20000 [02:03<1:05:05,  4.96it/s]


KeyboardInterrupt: ignored

**Train model using fake data**

In [ ]:
n_classes = 100
source = torch.rand(size=(32, 16, 9))
target_in = torch.rand(size=(32, 16, 8))
target_out = torch.rand(size=(32, 16, 1))

ts = TimeSeriesForcasting(n_encoder_inputs=9, n_decoder_inputs=8)

pred = ts((source, target_in))

print(pred.size())

ts.training_step((source, target_in, target_out), batch_idx=1)


torch.Size([32, 16, 1])


/usr/local/lib/python3.7/site-packages/pytorch_lightning/core/lightning.py:373: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  "You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet."


tensor(2., grad_fn=<MeanBackward0>)